# Implementation of the Kuribayashi BERT minus model

## libraries

In [1]:
!pip install transformers --upgrade
!pip install ipywidgets
!pip install IProgress
!pip install datasets
!pip install torch-lr-finder

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 5.8 MB 28.7 MB/s eta 0:00:01
     |████████████████████████████████| 7.6 MB 92.7 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.5
    Uninstalling transformers-4.12.5:
      Successfully uninstalled transformers-4.12.5
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertModel, BertForSequenceClassification
from transformers import BatchEncoding, default_data_collator, DataCollatorWithPadding

import torch
import torch.nn as nn

import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import datasets
from datasets import load_metric

from torch.utils.data import DataLoader

from tqdm import tqdm
from operator import itemgetter

In [3]:
print(transformers.__version__)

4.25.1


## tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

## data

In [6]:
DATA_FILE = '/notebooks/KURI-BERT/notebooks/full_formula_w_fts/pe_dataset_for_bert_minus_w_fts_combined_corrected.pt'

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load data

In [8]:
dataset = torch.load(DATA_FILE)

### preprocessing

In [13]:
MAX_LENGTH = 0
MAX_SPAN = 0

for split in ['train', 'test', 'validation']:
    
    for col_name in ['fts_spans']:
        
        for x in dataset[split][col_name]:
            
            
            if max(x,key=itemgetter(1))[1] > MAX_SPAN:
                
                MAX_SPAN = max(x,key=itemgetter(1))[1]
                MAX_SPAN = min(MAX_SPAN, tokenizer.model_max_length - 2)
            
            if len(x) > MAX_LENGTH:
                
                MAX_LENGTH = len(x)

In [17]:
def get_padding(batch, padding_target):    
    
    if padding_target == 'am_spans':
        
        col_name = 'am_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'ac_spans':
        
        col_name = 'ac_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'fts_spans':
        
        col_name = 'fts_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'label':
    
        col_name = 'paragraph_labels_numeric'
        padding_val = [-100]      
        max_length = MAX_LENGTH

    padded_spans = []

    for idx, span in enumerate(batch[col_name]):

        padded_span = batch[col_name][idx] + (max_length - len(span)) * padding_val
        padded_spans.append(padded_span)

    return padded_spans         

In [18]:
def get_combined_spans(am_spans_ll, ac_spans_ll, fts_spans_ll):
    
    spans_ll = []
    
    for am_spans, ac_spans, fts_spans in zip(am_spans_ll, ac_spans_ll, fts_spans_ll):
        
        spans = []
        
        for am_span, ac_span, fts_span in zip(am_spans, ac_spans, fts_spans):
            
            for idx in [0,1]:
                
                if am_span[idx] > MAX_SPAN:
                    am_span[idx] = MAX_SPAN
                if ac_span[idx] > MAX_SPAN:
                    ac_span[idx] = MAX_SPAN
                if fts_span[idx] > MAX_SPAN:
                    fts_span[idx] = MAX_SPAN

            span = [am_span, ac_span, fts_span]
            spans.extend(span)
            
        spans_ll.append(spans)

    return spans_ll

### tokenize 

In [20]:
def tokenize(batch):
    
    tokenized_text = tokenizer(batch['full_paragraph'], truncation=True, padding=True, max_length=512)
    tokenized_text['label'] = get_padding(batch, 'label')
    tokenized_text['am_spans'] = get_padding(batch, 'am_spans')
    tokenized_text['ac_spans'] = get_padding(batch, 'ac_spans')
    tokenized_text['fts_spans'] = get_padding(batch, 'fts_spans')
    tokenized_text['spans'] = get_combined_spans(tokenized_text['am_spans'], tokenized_text['ac_spans'], tokenized_text['fts_spans'])      
    
    return tokenized_text

In [ ]:
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset['train']))

In [30]:
dataset['test'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")
dataset['train'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")
dataset['validation'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")

In [31]:
dataset = dataset.map(lambda batch: batch, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'spans', 'label'])

## span representation function

In [33]:
def minus_one(t):
    
    return torch.where(t == 0, 0, t-1)

def plus_one(t):
    
    return torch.where(t >= MAX_SPAN, MAX_SPAN, t+1)

In [34]:
def get_span_representations(outputs, spans):

    
    batch_size = spans.shape[0]
    
    nr_span_indices = spans.shape[1]
    
    
    idx_l_ams = range(0, nr_span_indices, 3)
    idx_l_acs = range(1, nr_span_indices, 3)
    idx_l_fts = range(2, nr_span_indices, 3)
    
    # Add 1 to all span indices (both am and ac) to offset for the CLS token in the input_ids.
    
    am_spans = spans[:, idx_l_ams, :] + 1 
    ac_spans = spans[:, idx_l_acs, :] + 1
    fts_spans = spans[:, idx_l_fts, :] + 1
    
    
    am_spans_minus_one = minus_one(am_spans)
    am_spans_plus_one = plus_one(am_spans)
    
    ac_spans_minus_one = minus_one(ac_spans)
    ac_spans_plus_one = plus_one(ac_spans)
    
    fts_spans_minus_one = minus_one(fts_spans)
    fts_spans_plus_one = plus_one(fts_spans)
    
    
    am_spans = am_spans.flatten(start_dim=1)
    ac_spans = ac_spans.flatten(start_dim=1)
    fts_spans = fts_spans.flatten(start_dim=1)
    
    nr_adus = ac_spans.shape[1] // 2
    
    
    am_spans_minus_one = am_spans_minus_one.flatten(start_dim=1)
    am_spans_plus_one = am_spans_plus_one.flatten(start_dim=1)
    
    ac_spans_minus_one = ac_spans_minus_one.flatten(start_dim=1)
    ac_spans_plus_one = ac_spans_plus_one.flatten(start_dim=1)
    
    fts_spans_minus_one = fts_spans_minus_one.flatten(start_dim=1)
    fts_spans_plus_one = fts_spans_plus_one.flatten(start_dim=1)
    
    ############# FOR AMs #################
    
    outputs_am = outputs[:,am_spans,:]
    outputs_am = torch.cat([outputs_am[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am = outputs_am.reshape(batch_size, nr_adus * 2, -1)
    
    
    outputs_am_minus_one = outputs[:,am_spans_minus_one,:]
    outputs_am_minus_one = torch.cat([outputs_am_minus_one[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am_minus_one = outputs_am_minus_one.reshape(batch_size, nr_adus * 2, -1)
    
    outputs_am_plus_one = outputs[:,am_spans_plus_one,:]
    outputs_am_plus_one = torch.cat([outputs_am_plus_one[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am_plus_one = outputs_am_plus_one.reshape(batch_size, nr_adus * 2, -1)
    
    
    
    ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
    # ============== the corrected 1st one =================== 
    
    outputs_am_first_term = torch.cat([outputs_am[:,i+1,:] - outputs_am_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1) # i + 1 here means j in kuri, i here means i in kuri
    outputs_am_first_term = outputs_am_first_term.reshape(batch_size, -1, 768)
    
     # ============== the corrected 2nd one ==================
    
    outputs_am_second_term = torch.cat([outputs_am[:,i,:] - outputs_am_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
    outputs_am_second_term = outputs_am_second_term.reshape(batch_size, -1, 768)
    
    
        # ============== the corrected third one ================== 
    
    outputs_am_third_term = torch.cat([outputs_am_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_am_third_term = outputs_am_third_term.reshape(batch_size, -1, 768)
    

        # ============== the corrected fourth one ==================
    
    outputs_am_fourth_term = torch.cat([outputs_am_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
    outputs_am_fourth_term = outputs_am_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    am_minus_representations = torch.cat([outputs_am_first_term, outputs_am_second_term, outputs_am_third_term, outputs_am_fourth_term], dim=-1)   
    
    
    
    ### am minus span representation according to kuribayashi paper is now above.
    
    
    outputs_ac = outputs[:,ac_spans,:]
    outputs_ac = torch.cat([outputs_ac[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_ac = outputs_ac.reshape(batch_size, nr_adus * 2, -1)
    
    
    outputs_ac_minus_one = outputs[:,ac_spans_minus_one,:] # xxx. added to solve bug 2
    outputs_ac_minus_one = torch.cat([outputs_ac_minus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_ac_minus_one = outputs_ac_minus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    outputs_ac_plus_one = outputs[:,ac_spans_plus_one,:] # xxx. added to solve bug 2
    outputs_ac_plus_one = torch.cat([outputs_ac_plus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_ac_plus_one = outputs_ac_plus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    ### Now that we have outputs_ac i.e. outputs at ac_span indices, now create the four Kuri forumlas for ACs
    
    
    # ============== the corrected first one ===================
    
    outputs_ac_first_term = torch.cat([outputs_ac[:,i+1,:] - outputs_ac_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_ac_first_term = outputs_ac_first_term.reshape(batch_size, -1, 768)
    
        # ============== the correct second one ==================
    
    outputs_ac_second_term = torch.cat([outputs_ac[:,i,:] - outputs_ac_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run
    outputs_ac_second_term = outputs_ac_second_term.reshape(batch_size, -1, 768)
    
    
    # ============== the corrected third term ==================
    
    outputs_ac_third_term = torch.cat([outputs_ac_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_ac_third_term = outputs_ac_third_term.reshape(batch_size, -1, 768)

    
     # ============== the corrected fourth term ================== xxx. for .788
    
    outputs_ac_fourth_term = torch.cat([outputs_ac_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run
    outputs_ac_fourth_term = outputs_ac_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    ac_minus_representations = torch.cat([outputs_ac_first_term, outputs_ac_second_term, outputs_ac_third_term, outputs_ac_fourth_term], dim=-1)   
    
    
    
    ### ac minus span representation according to kuribayashi paper is now above.
    
    
     ############# FOR Fts #################
    
    outputs_fts = outputs[:,am_spans,:] # am spans for checking
    outputs_fts = torch.cat([outputs_fts[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_fts = outputs_fts.reshape(batch_size, nr_adus * 2, -1)
    
    
    
    outputs_fts_minus_one = outputs[:,fts_spans_minus_one,:] 
    outputs_fts_minus_one = torch.cat([outputs_fts_minus_one[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_fts_minus_one = outputs_fts_minus_one.reshape(batch_size, nr_adus * 2, -1)
    
    outputs_fts_plus_one = outputs[:,fts_spans_plus_one,:]
    outputs_fts_plus_one = torch.cat([outputs_fts_plus_one[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_fts_plus_one = outputs_fts_plus_one.reshape(batch_size, nr_adus * 2, -1)
    
    
    
    ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
    # ============== (batch_size x nr_adus x 768) =================== 
    
    outputs_fts_first_term = torch.cat([outputs_fts[:,i+1,:] - outputs_fts_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1) # i + 1 here means j in kuri, i here means i in kuri
    outputs_fts_first_term = outputs_fts_first_term.reshape(batch_size, -1, 768)
    
     # ============== the corrected 2nd one ==================
    
    outputs_fts_second_term = torch.cat([outputs_fts[:,i,:] - outputs_fts_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_second_term = outputs_fts_second_term.reshape(batch_size, -1, 768)
    
        # ============== the corrected third one ================== 
    
    outputs_fts_third_term = torch.cat([outputs_fts_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_third_term = outputs_fts_third_term.reshape(batch_size, -1, 768)
    

        # ============== the corrected fourth one ==================
    
    outputs_fts_fourth_term = torch.cat([outputs_fts_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_fourth_term = outputs_fts_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    fts_minus_representations = torch.cat([outputs_fts_first_term, outputs_fts_second_term, outputs_fts_third_term, outputs_fts_fourth_term], dim=-1)   
    
    
    
    ### fts minus span representation according to kuribayashi paper is now above.    

    
    return am_minus_representations, ac_minus_representations, fts_minus_representations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

### span representation function old

## custom BERT model

In [35]:
class CustomBERTKuri(nn.Module):

    def __init__(self, first_model, model_am, model_ac, model_fts, nr_classes):
        
        super(CustomBERTKuri, self).__init__()
        
        self.first_model = first_model
        
        self.intermediate_linear_am = nn.Linear(3072, 768)
        self.intermediate_linear_ac = nn.Linear(3072, 768) 
        self.intermediate_linear_fts = nn.Linear(3072, 768)
        
        self.model_am = model_am
        self.model_ac = model_ac
        self.model_fts = model_fts
        
        self.nr_classes = nr_classes
                
        self.fc = nn.Linear(self.model_am.config.hidden_size + self.model_ac.config.hidden_size + self.model_fts.config.hidden_size, self.nr_classes)

    def forward(self, inputs):
        
        batch_tokenized, batch_spans = inputs 
        outputs = self.first_model(batch_tokenized, output_hidden_states=True)[1][12]
        am_minus_representations, ac_minus_representations, fts_minus_representations = get_span_representations(outputs, batch_spans)


        am_minus_representations = self.intermediate_linear_am(am_minus_representations)
        ac_minus_representations = self.intermediate_linear_ac(ac_minus_representations)
        fts_minus_representations = self.intermediate_linear_fts(fts_minus_representations)


        output_model_am = self.model_am(inputs_embeds = am_minus_representations)[0]
        output_model_ac = self.model_ac(inputs_embeds = ac_minus_representations)[0]
        output_model_fts = self.model_fts(inputs_embeds = fts_minus_representations)[0]

        adu_representations = torch.cat([output_model_am, output_model_ac, output_model_fts], dim=-1)
        output = self.fc(adu_representations)

        return output

## Run

In [36]:
NB_EPOCHS = 40
BATCH_SIZE = 28

In [ ]:
first_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
first_model.load_state_dict(torch.load('/notebooks/KURI-BERT/notebooks/full_formula_w_fts/icann_finetuned_work/finetuned_model.pth'))

In [38]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [39]:
model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [40]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [41]:
custom_model = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 3)

In [ ]:
custom_model.to(device)

In [43]:
loss = nn.CrossEntropyLoss(ignore_index=- 100)

In [44]:
optimizer = torch.optim.AdamW(custom_model.parameters(), lr=0.0035111917342151295)

In [46]:
NR_BATCHES = len(dataset['train']) / BATCH_SIZE
num_training_steps = NB_EPOCHS * NR_BATCHES
num_warmup_steps = int(0.2 * num_training_steps)

In [47]:
def lr_lambda(current_step: int):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

In [48]:
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

### create dataloaders

In [49]:
train_dataloader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=True)

In [51]:
def flatten_list(list_of_lists):
    return [x for sublist in list_of_lists for x in sublist]

In [52]:
def remove_dummy_labels(test_preds, test_labels):
    
    idxes = []
    test_labels_l = []
    for idx, val in enumerate(test_labels):
        if val != -100:
            idxes.append(idx)
            test_labels_l.append(val)
    
    test_preds_l = []
    for idx, val in enumerate(test_preds):
        for good_idx in idxes:
            if idx == good_idx:
                test_preds_l.append(val)
        
    return test_preds_l, test_labels_l

## training 

In [55]:
def train(model, loss=None, optimizer=None, train_dataloader=None, val_dataloader=None, nb_epochs=20):
    """Training loop"""

    min_f1 = -torch.inf
    train_losses = []
    val_losses = []

    # Iterrate over epochs
    for e in range(nb_epochs):

        # Training
        train_loss = 0.0

        for i, batch in enumerate(tqdm(train_dataloader)):            
            
            #print(i)
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Reset gradients to 0
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(inputs)
            
            # Compute training loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            train_loss += current_loss.detach().item()

            # Compute gradients
            current_loss.backward()

            # Update weights
            optimizer.step()            
            
            del batch
        
        scheduler.step()
            
        
        # Validation
        val_loss = 0.0
        
        # torch.cuda.empty_cache()
        
        # Put model in eval mode
        model.eval()
        
        preds_l = []
        labels_l = []
        
        for batch in tqdm(val_dataloader):            
            
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Forward Pass
            outputs = model(inputs)

            # Compute validation loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            val_loss += current_loss.detach().item()
            
            preds_for_f1 = torch.argmax(outputs, dim=2).flatten().tolist()
            labels_for_f1 = labels.flatten().tolist()
            
            preds_l.append(preds_for_f1)
            labels_l.append(labels_for_f1)
            
            del batch
        
        # Prints
        
        preds_l = flatten_list(preds_l)
        labels_l = flatten_list(labels_l)
        
        preds_l, labels_l = remove_dummy_labels(preds_l, labels_l)
        
        f1_score_epoch = f1_score(preds_l, labels_l, average='macro')        
        
        print(f"Epoch {e+1}/{nb_epochs} \
                \t Training Loss: {train_loss/len(train_dataloader):.3f} \
                \t Validation Loss: {val_loss/len(val_dataloader):.3f} \
                \t F1 score: {f1_score_epoch}")
        
        train_losses.append(train_loss/len(train_dataloader))
        val_losses.append(val_loss/len(val_dataloader))
        

        # Save model if val loss decreases
        if f1_score_epoch > min_f1:

            min_f1 = f1_score_epoch
            torch.save(model.first_model.state_dict(), 'first_model.pt')
            torch.save(model.model_am.state_dict(), 'model_am.pt')
            torch.save(model.model_ac.state_dict(), 'model_ac.pt')
            torch.save(model.model_fts.state_dict(), 'model_fts.pt')
            torch.save(model.state_dict(), 'best_model.pt')
            
    return train_losses, val_losses

In [60]:
train_losses, val_losses = train(custom_model, loss, optimizer, train_dataloader, val_dataloader, NB_EPOCHS)

100%|██████████| 10/10 [00:03<00:00,  2.82it/s]


Epoch 1/40                 	 Training Loss: 1.523                 	 Validation Loss: 1.572                 	 F1 score: 0.1237552502908713


100%|██████████| 10/10 [00:03<00:00,  2.88it/s]


Epoch 2/40                 	 Training Loss: 0.992                 	 Validation Loss: 0.521                 	 F1 score: 0.7083694893913872


100%|██████████| 10/10 [00:03<00:00,  2.87it/s]


...

Epoch 38/40                 	 Training Loss: 0.917                 	 Validation Loss: 0.890                 	 F1 score: 0.2613299449385853


100%|██████████| 10/10 [00:03<00:00,  2.88it/s]


Epoch 39/40                 	 Training Loss: 0.923                 	 Validation Loss: 0.894                 	 F1 score: 0.2613299449385853


100%|██████████| 10/10 [00:03<00:00,  2.90it/s]

Epoch 40/40                 	 Training Loss: 0.920                 	 Validation Loss: 0.890                 	 F1 score: 0.2613299449385853


### Predictions

In [62]:
test_dataloader = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
first_model =  BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
first_model.load_state_dict(torch.load('first_model.pt'))

In [65]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_am.load_state_dict(torch.load('model_am.pt'))

<All keys matched successfully>

In [66]:
model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_ac.load_state_dict(torch.load('model_ac.pt'))

<All keys matched successfully>

In [67]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_fts.load_state_dict(torch.load('model_fts.pt'))

<All keys matched successfully>

In [ ]:
# Load best model

custom_model_2 = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 3)
custom_model_2.load_state_dict(torch.load('best_model.pt'))

custom_model_2.to(device).eval()

In [69]:
def predict(model, test_dataloader=None):
    
    """Prediction loop"""

    preds_l = []
    labels_l = []
    
    model.eval()

    for batch in test_dataloader:            
            
        # unpack batch             
        labels = batch['label'].to(device).flatten().tolist()
        spans = batch['spans'].to(device)
        input_ids = batch['input_ids'].to(device)
        
        inputs = input_ids, spans

        # get output
        
        raw_preds = model(inputs).to('cpu')
        # print(raw_preds.shape)
        raw_preds = raw_preds.detach()#.numpy()

        # Compute argmax
        
        predictions = torch.argmax(raw_preds, dim=2).flatten().tolist()
        preds_l.append(predictions)
        labels_l.append(labels)        
        
        del batch
            
    return flatten_list(preds_l), flatten_list(labels_l)

In [76]:
test_preds, test_labels = predict(custom_model_2, test_dataloader)

In [77]:
print(classification_report(test_labels, test_preds, digits=3))

              precision    recall  f1-score   support

        -100      0.000     0.000     0.000      3033
           0      0.678     0.897     0.772       155
           1      0.095     0.663     0.166       303
           2      0.375     0.916     0.532       805

    accuracy                          0.251      4296
   macro avg      0.287     0.619     0.367      4296
weighted avg      0.101     0.251     0.139      4296



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [78]:
test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [79]:
print(classification_report(test_labels_l, test_preds_l, digits=3))

              precision    recall  f1-score   support

           0      0.903     0.897     0.900       155
           1      0.713     0.663     0.687       303
           2      0.891     0.916     0.903       805

    accuracy                          0.853      1263
   macro avg      0.836     0.825     0.830      1263
weighted avg      0.850     0.853     0.851      1263

